In [2]:
from tqdm.auto import tqdm

In [3]:
import ctranslate2
import transformers
from pathlib import Path

translator = ctranslate2.Translator("madlad400-7b-mt-bt-ct2-int8_float16", compute_type="auto")
tokenizer = transformers.AutoTokenizer.from_pretrained("jbochi/madlad400-7b-mt-bt")

In [4]:
def translate(input_text):
    prefix = "<2en> "
    input_text = str(input_text).replace('\n','')
    input_tokens  = tokenizer.convert_ids_to_tokens(tokenizer.encode(prefix + input_text))

    results = translator.translate_batch([input_tokens])

    output_tokens = results[0].hypotheses[0]
    output_text = tokenizer.decode(tokenizer.convert_tokens_to_ids(output_tokens))
    text_start = '\n' if input_text[0] == '\n' else ' '
    return text_start+output_text

In [5]:
def handle_items(items):
    if len(items.children) != 0:
        for i in items.children:
            handle_items(i)
    if items.name == 'item':
        for i in range(len(items.text)):
            try:
                if items[i].text == '\\\\':
                    continue
            except IndexError:
                continue
            if hasattr(items[i], 'name'):
                continue
            items[i].text = translate(items[i].text)



In [6]:
def handle_text(node):
    name = node.name
    if name in ['chapter','section','text']:
        if len(node.text) == 0:
            return
        if node.text[0][0] == '%':
            return
        print(node.text)
        node[0].text = '\n'+translate(node[0].text)
    if name == 'figure':
        if len(node.caption.text) == 1:
            node.caption.string = translate(node.caption.string)
        else:
            node.caption[0].text = translate(node.caption[0].text)

    if name == "itemize":
        handle_items(node)

    if len(name) > 300:
        node.name = translate(node.name)

In [7]:
with open(Path('Diplomaterv/content/1_irodalom.tex'),'r',encoding='utf-8') as f:
    text = f.read()

In [8]:
from TexSoup import TexSoup
soup = TexSoup(text, tolerance=1)

In [9]:
for node in tqdm(soup.all):
    handle_text(node)

  0%|          | 0/95 [00:00<?, ?it/s]

['Előzmények']
['Földmegfigyelés']
['\nA földmegfigyelés feladata, hogy bolygónk felületéről minél pontosabb képet alkosson különböző technológiák segítségével. A legelterjedtebb módszer földmegfigyelésre különböző távérzékelési technológiák alkalmazása, a dolgozatomban kizárólag ilyen technológiákkal fogok foglalkozni. A telekommunikációs- és űrtechnológiák fejlődésével párhuzamosan egyre több műhold pásztázza a Földet, ami növeli a rendelkezésre álló adatok mennyiségét. Ezt az adatmennyiséget tovább növeli a drónok és egyéb olcsón elérhető repülő eszközök elterjedése, ami a földmegfigyeléshez használható nagy felbontású képek olcsó és gyors előállítását teszi lehetővé.\nEzeknek az adatforrásoknak a rohamos növekedése számos informatikai és adatkezelési kihívás elé állította a földmegfigyeléssel foglalkozó kutatókat és szakembereket, emiatt számos technológiai újítás korai felhasználója a szakterület. Az így keletkező adattömegek segítségével különböző applikációk készíthetőek, melyek

In [10]:
with open(Path('Diplomaterv/content/1_irodalom_eng.tex'),'w',encoding='utf-8') as f:
    f.write(str(soup))